In [8]:
import pandas as pd
from collections import Counter as ctr

In [3]:
#make a dataframe ignoring the last one from train.txt
train = pd.read_csv('train.txt', sep=' ', usecols=[0,1], names=['word','pos'] )
train['word'] = train.word.str.lower()
train.head(10)

,word,pos
0,confidence,NN
1,in,IN
2,the,DT
3,pound,NN
4,is,VBZ
5,widely,RB
6,expected,VBN
7,to,TO
8,take,VB
9,another,DT


In [4]:
train['ppos'] = train.pos.shift(1)
train['ppos'] = train['ppos'].fillna('O')
smooth = 0.00000001
train.head()

,word,pos,ppos
0,confidence,NN,O
1,in,IN,NN
2,the,DT,IN
3,pound,NN,DT
4,is,VBZ,NN


### Write the functions
$P(T{i}|T{i-1})$
$P(W{i}|T{i})$

In [20]:
pos_ppos = {} #contains a dict for every ppos in it... shows the amount of times each pos follows the ppos(key)
pos_ppos_denom = {} #count of how many times a ppos shows up
for col_name in set(train.ppos):
    sub_frame = train[train.ppos == col_name] #dataframe with all instances of given ppos
    counted = ctr(sub_frame.pos) #dict with counts for each pos following ppos
    pos_ppos[col_name] = counted
    pos_ppos_denom[col_name] = len(sub_frame)
    break
#print(pos_ppos)
#print(counted)



{'RB': Counter({'IN': 854, 'VB': 725, 'JJ': 679, ',': 616, 'VBN': 570, '.': 413, 'RB': 389, 'DT': 335, 'VBD': 334, 'VBZ': 271, 'VBP': 206, 'VBG': 203, 'TO': 168, 'CD': 151, 'JJR': 115, '$': 101, 'MD': 76, 'PRP': 74, 'NN': 53, 'CC': 47, 'NNP': 45, 'NNS': 33, 'RBR': 31, ':': 25, 'WRB': 23, 'PRP$': 18, 'WP': 11, '``': 7, "''": 7, 'EX': 4, 'JJS': 4, 'POS': 3, 'PDT': 3, 'WDT': 3, '#': 2, 'RBS': 2, 'NNPS': 2, '(': 1, ')': 1, 'SYM': 1, 'FW': 1})}


In [ ]:
# pos_ppos = {}

# for col_name in set(test.ppos):
#     sub_frame = train
# def Ptt(Ti,prev)

# if Ti not in pos_ppos[Tprev] return smooth (.000000001)

## Build the trellis

## Evaluate: decode

In [14]:
test = pd.read_csv('test.txt', sep=' ', usecols=[0,1], names=['word','pos'])
test['word'] = test['word'].str.lower()
test.head()

,word,pos
0,rockwell,NNP
1,international,NNP
2,corp.,NNP
3,'s,POS
4,tulsa,NNP
